In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver
load_dotenv()

True

In [2]:
llm = HuggingFaceEndpoint(
	model="openai/gpt-oss-120b",
	task="text-generation",
)
model = ChatHuggingFace(llm=llm)

In [3]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [4]:
def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt)

    return {
        'joke': response.content
    }

In [5]:
def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]} in 2-3 lines'
    response = model.invoke(prompt)

    return {
        'explanation': response.content 
    }

In [6]:
graph = StateGraph(JokeState)

graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)

graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [7]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1) # type: ignore

{'topic': 'pizza',
 'joke': "Why did the slice of pizza start a band?  \n\nBecause it already had the perfect “crust” and “cheese” for a hit single—“Dough‑n't Stop Believin’!” 🍕😄",
 'explanation': "The punchline plays on two puns: “crust” sounds like “must,” and “cheese” evokes “cheesy” (catchy) music, suggesting the pizza already has the essential ingredients for a hit song. The title “Dough‑n't Stop Believin’” riffs on the famous track “Don’t Stop Believin’,” swapping “don’t” for “dough” to keep the pizza theme. Together, the wordplay turns a simple slice into a ready‑made rock band."}

In [8]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza start a band?  \n\nBecause it already had the perfect “crust” and “cheese” for a hit single—“Dough‑n't Stop Believin’!” 🍕😄", 'explanation': "The punchline plays on two puns: “crust” sounds like “must,” and “cheese” evokes “cheesy” (catchy) music, suggesting the pizza already has the essential ingredients for a hit song. The title “Dough‑n't Stop Believin’” riffs on the famous track “Don’t Stop Believin’,” swapping “don’t” for “dough” to keep the pizza theme. Together, the wordplay turns a simple slice into a ready‑made rock band."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4f54-6db7-8002-5905da6e0b0c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-19T12:26:07.591147+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4905-6293-8001-fac5af5b9ae9'}}, tasks=(), interrupts=

In [9]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza start a band?  \n\nBecause it already had the perfect “crust” and “cheese” for a hit single—“Dough‑n't Stop Believin’!” 🍕😄", 'explanation': "The punchline plays on two puns: “crust” sounds like “must,” and “cheese” evokes “cheesy” (catchy) music, suggesting the pizza already has the essential ingredients for a hit song. The title “Dough‑n't Stop Believin’” riffs on the famous track “Don’t Stop Believin’,” swapping “don’t” for “dough” to keep the pizza theme. Together, the wordplay turns a simple slice into a ready‑made rock band."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4f54-6db7-8002-5905da6e0b0c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-19T12:26:07.591147+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4905-6293-8001-fac5af5b9ae9'}}, tasks=(), interrupts

In [10]:
config2 = {"configurable": {"thread_id": "2"}}
workflow.invoke({'topic':'pasta'}, config=config2)

{'topic': 'pasta',
 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to *pasta* test! 🍝😄',
 'explanation': 'The punchline plays on the word “pasta,” the Italian staple that spaghetti is made of, sounding like “passed.” In a workplace, “passed test” is a reason for a promotion, so the joke pretends the noodle earned a raise because it “pasta‑tested” (i.e., passed the test). The pun and the visual of a noodle getting a promotion make it funny.'}

In [11]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza start a band?  \n\nBecause it already had the perfect “crust” and “cheese” for a hit single—“Dough‑n't Stop Believin’!” 🍕😄", 'explanation': "The punchline plays on two puns: “crust” sounds like “must,” and “cheese” evokes “cheesy” (catchy) music, suggesting the pizza already has the essential ingredients for a hit song. The title “Dough‑n't Stop Believin’” riffs on the famous track “Don’t Stop Believin’,” swapping “don’t” for “dough” to keep the pizza theme. Together, the wordplay turns a simple slice into a ready‑made rock band."}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4f54-6db7-8002-5905da6e0b0c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-19T12:26:07.591147+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-4905-6293-8001-fac5af5b9ae9'}}, tasks=(), interrupts=

In [12]:
list(workflow.get_state_history(config2))

[StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to *pasta* test! 🍝😄', 'explanation': 'The punchline plays on the word “pasta,” the Italian staple that spaghetti is made of, sounding like “passed.” In a workplace, “passed test” is a reason for a promotion, so the joke pretends the noodle earned a raise because it “pasta‑tested” (i.e., passed the test). The pun and the visual of a noodle getting a promotion make it funny.'}, next=(), config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-5d92-6334-8002-64cbaa120b74'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-19T12:26:09.084258+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-57d2-6371-8001-8004f8e1610a'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'pasta', 'joke': 'Why did the spaghetti get a promotion?  \n\nBecause it knew how to 

In [13]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f06cc6e-7232-6cb1-8000-f71609e6cec5"}})

StateSnapshot(values={}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f06cc6e-7232-6cb1-8000-f71609e6cec5'}}, metadata=None, created_at=None, parent_config=None, tasks=(), interrupts=())

In [14]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0dbf45-84f4-6f4b-8000-d795d6888bbe", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0dcd5e-5e0d-6c92-8000-ba2f19f39a71'}}

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5e-5e0d-6c92-8000-ba2f19f39a71'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-12-19T12:26:09.134908+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dbf45-84f4-6f4b-8000-d795d6888bbe'}}, tasks=(PregelTask(id='78018478-c553-1726-2cbc-0c41d1ad760e', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the slice of pizza start a band?  \n\nBecause it already had the perfect “crust” and “cheese” for a hit single—“Dough‑n't Stop Believin’!” 🍕😄", 'explanation': "The punchline plays on two puns: “crust” sounds like “must,” and “cheese” evokes “cheesy” (catchy) music, suggesting the pizza already has the essential ingredien

In [17]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0dcd5e-5e0d-6c92-8000-ba2f19f39a71"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa go to therapy?  \n\nBecause it was feeling a little *crust*y on the outside but totally *filled* with anxiety on the inside!',
 'explanation': 'The joke plays on the literal parts of a samosa—its **crust** (the fried outer shell) and its **filling**—and uses them as metaphors for emotional states. “Crust‑y on the outside” suggests a tough, perhaps defensive exterior, while “filled with anxiety on the inside” twists the word *filled* (the tasty interior) to mean the samosa is mentally overwhelmed, making the punchline a humorous blend of food description and therapy talk.'}

In [ ]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa', 'joke': 'Why did the samosa go to therapy?  \n\nBecause it was feeling a little *crust*y on the outside but totally *filled* with anxiety on the inside!', 'explanation': 'The joke plays on the literal parts of a samosa—its **crust** (the fried outer shell) and its **filling**—and uses them as metaphors for emotional states. “Crust‑y on the outside” suggests a tough, perhaps defensive exterior, while “filled with anxiety on the inside” twists the word *filled* (the tasty interior) to mean the samosa is mentally overwhelmed, making the punchline a humorous blend of food description and therapy talk.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5f-0d81-661e-8002-c62eac02ce70'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-12-19T12:26:27.532324+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0dcd5f-06d9-639c-8001-f37